# Train Legal Classifiers

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

In [0]:
from johnsnowlabs import * 

# Multilabel classifier training

## Loading the data

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Legal/data/finance_data.csv

dbutils.fs.cp("file:/databricks/driver/finance_data.csv", "dbfs:/") 

Out[2]: True

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/finance_data.csv')
df['label'] = df['label'].apply(eval)

In [0]:
data = spark.createDataFrame(df)

# If you have a single dataset, then split it or else you can load the test dataset the same way that you load the train data.
train, test = data.randomSplit([0.8, 0.2], seed = 123)

In [0]:
train.show(truncate=50)

+--------------------------------------------------+-----------------------------+
 provision| label|
+--------------------------------------------------+-----------------------------+
(a) Consultant or Company may terminate this Pr...| [terminations]|
(a) Effective as of the Effective Date, the Hol...| [waivers, terminations]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Agent or any Len...| [waivers, amendments]|
(a) No failure or delay of the Administrative A...| [waivers, amendments]|
(a) The Credit Agreement is, effective as of th...| [amendments]|
(a) The provisions of this Agreement shall be b...| [assigns, successors]|
(a) To induce the other parties hereto to enter...|[representations, warranties]|
(a)  The provisions of this Agreement shall be ...| [assigns, successors]|
(a) All of the representations and warranties m...|[representations, warranties]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Collateral Agent...| [waivers, amendments]|
(a) No failure or delay of the Agents or any Le...| [waivers, amendments]|
(a) THIS AGREEMENT AND EACH OTHER LOAN DOCUMENT...| [governing laws]|
(a) THIS AGREEMENT WAS NEGOTIATED IN THE STATE ...| [governing laws]|
(a) The provisions of this Agreement shall be b...| [assigns, successors]|
(a) This Agreement may be amended, supplemented...| [waivers, amendments]|
(a) To induce the other parties hereto to enter...|[representations, warranties]|
(a)  The Performance Guarantor hereby waives pr...| [waivers]|
+--------------------------------------------------+-----------------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import col

test.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 [governing laws]| 727|
 [counterparts]| 569|
 [notices]| 556|
 [entire agreements]| 556|
 [severability]| 496|
 [survival]| 358|
[assigns, success...| 319|
 [amendments]| 319|
 [expenses]| 248|
 [terminations]| 224|
[waivers, amendme...| 198|
[representations,...| 191|
 [waivers]| 186|
 [assignments]| 186|
 [representations]| 97|
[amendments, enti...| 73|
 [successors]| 64|
 [warranties]| 40|
[amendments, term...| 27|
[assignments, suc...| 10|
+--------------------+-----+
only showing top 20 rows

## With Universal Encoder

In [0]:
document_assembler = nlp.DocumentAssembler() \
      .setInputCol("provision") \
      .setOutputCol("document") \
      .setCleanupMode("shrink")

embeddings = nlp.UniversalSentenceEncoder.pretrained() \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

classsifierdl = nlp.MultiClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("label")\
      .setMaxEpochs(30)\
      .setEnableOutputLogs(True)

clf_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        embeddings,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 564 ms, sys: 216 ms, total: 781 ms
Wall time: 2min 43s

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','provision',"class.result").toPandas()
preds_df.head()

Out[10]:

,label,provision,result
0,"[waivers, amendments]",(a) No failure or delay by any Agent or any Le...,"[waivers, amendments]"
1,[assignments],"(a) Seller, the Agent, each Managing Agent, ea...","[successors, assignments]"
2,[waivers],(a) Any provision of this Agreement may be wai...,"[waivers, amendments]"
3,[notices],(a) Except where telephonic instructions or no...,[notices]
4,"[governing laws, entire agreements]","(a) THIS AGREEMENT AND ANY CLAIM, CONTROVERSY,...","[governing laws, entire agreements]"


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(preds_df['label'])
y_pred = mlb.fit_transform(preds_df['result'])


print("Classification report: \n", (classification_report(y_true, y_pred)))
print("F1 micro averaging:",(f1_score(y_true, y_pred, average='micro')))
print("ROC: ",(roc_auc_score(y_true, y_pred, average="micro")))


Classification report: 
 precision recall f1-score support

 0 0.89 0.80 0.84 643
 1 0.74 0.58 0.65 210
 2 0.79 0.87 0.83 333
 3 0.99 0.98 0.99 579
 4 0.98 0.94 0.96 667
 5 0.96 0.95 0.95 249
 6 0.98 0.98 0.98 747
 7 0.94 0.96 0.95 557
 8 0.86 0.82 0.84 288
 9 0.97 0.94 0.96 515
 10 0.85 0.91 0.88 407
 11 0.95 0.87 0.91 363
 12 0.88 0.75 0.81 256
 13 0.80 0.86 0.83 401
 14 0.81 0.74 0.77 231

 micro avg 0.91 0.89 0.90 6446
 macro avg 0.89 0.86 0.88 6446
weighted avg 0.91 0.89 0.90 6446
 samples avg 0.89 0.90 0.88 6446

F1 micro averaging: 0.900722588752749
ROC: 0.941155788922978

## With RoBerta Embeddings

We do not have have any specific Legal Sentence Embeddings, but we can use Legal RoBerta Embeddings and then average them.

In [0]:
embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setMaxSentenceLength(512)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("provision") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddingsSentence = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = nlp.MultiClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("label")\
    .setMaxEpochs(1)\
    .setEnableOutputLogs(True)

clf_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        embeddings,
        embeddingsSentence,
        classsifierdl
    ])

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('provision','label',"class.result").toPandas()

In [0]:
preds_df.head()

In [0]:
import os
log_file_name = os.listdir("/root/annotator_logs")[0]

with open("/root/annotator_logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(preds_df['label'])
y_pred = mlb.fit_transform(preds_df['result'])


print("Classification report: \n", (classification_report(y_true, y_pred)))
print("F1 micro averaging:",(f1_score(y_true, y_pred, average='micro')))
print("ROC: ",(roc_auc_score(y_true, y_pred, average="micro")))


### Saving & loading back the trained model

In [0]:
clf_pipelineModel.stages

In [0]:
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/MultilabelClfRoBerta')

In [0]:
# Load back  saved Multilabel Classifier Model
MultilabelClfModel = nlp.MultiClassifierDLModel.load('/dbfs/MultilabelClfRoBerta')

In [0]:
ld_pipeline = nlp.Pipeline(stages=[document_assembler, tokenizer, embeddings, embeddingsSentence, MultilabelClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([['']]).toDF("provision"))

In [0]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [0]:
ld_preds_df = ld_preds.select('provision','label',"class.result").toPandas()

In [0]:
ld_preds_df.head(10)

# Multiclass classifier training

## Loading the data

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/legal/data/finance_clf_data.csv

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/finance_clf_data.csv')

In [0]:
df.head()

In [0]:
df['label'].value_counts()

In [0]:
data = spark.createDataFrame(df)

train, test = data.randomSplit([0.8, 0.2], seed = 100)

In [0]:
from pyspark.sql.functions import col

train.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

In [0]:
from pyspark.sql.functions import col

test.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

## With Universal Encoder

In [0]:
document_assembler = nlp.DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document") 

embeddings = nlp.UniversalSentenceEncoder.pretrained() \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

classsifierdl = legal.ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("label")\
      .setMaxEpochs(30)\
      .setEnableOutputLogs(True)

clf_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        embeddings,
        classsifierdl
    ])

In [0]:
clf_pipelineModel = clf_pipeline.fit(train)

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','text',"class.result").toPandas()
preds_df.head()

In [0]:
# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))

### Saving & loading back the trained model

In [0]:
clf_pipelineModel.stages

In [0]:
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/Clf_Use')

In [0]:
# Load back  saved Classifier Model
ClfModel = legal.ClassifierDLModel.load('/dbfs/Clf_Use')

In [0]:
ld_pipeline = nlp.Pipeline(stages=[document_assembler, embeddings,ClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

In [0]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [0]:
ld_preds_df = ld_preds.select('text','label',"class.result").toPandas()

In [0]:
ld_preds_df.head()

## With RoBerta Embeddings

We do not have Legal Sentence Embeddings yet, But we can use the Legal RoBerta Embeddings and then average them.

In [0]:
embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setMaxSentenceLength(512)

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddingsSentence = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = legal.ClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("label")\
    .setMaxEpochs(8)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)

clf_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        embeddings,
        embeddingsSentence,
        classsifierdl
    ])

In [0]:
clf_pipelineModel = clf_pipeline.fit(train)

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','text',"class.result").toPandas()

In [0]:
preds_df.head()

In [0]:
log_files = os.listdir("/root/annotator_logs")

with open("/root/annotator_logs/"+log_files[0], "r") as log_file :
    print(log_file.read())

In [0]:
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))


# Save model and Zip it for Modelshub Upload/Downloads

In [0]:
# Save a Spark NLP model
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/ClfBert')

# cd into saved dir and zip
! cd /dbfs/ClfBert ; zip -r /dbfs/ClfBert.zip *